# Multi-Class Classification

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import ipywidgets
from ipywidgets import interact, interactive, interact_manual
import IPython
import pylab
pylab.rcParams['figure.figsize'] = (16, 8)

from utilities import plot_helpers
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn import datasets
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def build_confusion_matrix(pred_label, true_label, num_classes=2):
    """This works for predictions in {0, 1, ..., Num Classes}."""
    confusion_matrix = np.zeros((num_classes, num_classes))
    for row in range(num_classes):
        for col in range(num_classes):
            confusion_matrix[row, col] = np.sum(np.bitwise_and(pred_label == row, true_label == col))
    return confusion_matrix

def accuracy(confusion_matrix):
    return np.sum(np.diag(confusion_matrix)) / np.sum(confusion_matrix)

def precision(confusion_matrix):
    correct = np.diag(confusion_matrix)
    pred = np.sum(confusion_matrix, axis=1)
    pred[pred == 0] = 1  # to avoid nan.
    
    return np.min(correct / pred)

def recall(confusion_matrix):
    correct = np.diag(confusion_matrix)
    pred = np.sum(confusion_matrix, axis=0)
    pred[pred == 0] = 1  # to avoid nan.

    return np.min(correct / pred)

def f1_score(confusion_matrix):
    correct = np.diag(confusion_matrix)
    rec_dem = np.sum(confusion_matrix, axis=0)
    prec_dem = np.sum(confusion_matrix, axis=1)
        
    if np.any(correct == 0):
        return 0   # to avoid nan.
    else:
        return np.min(2 * correct / (rec_dem +  prec_dem))

def print_metrics(x, y, classifier):
    cm = build_confusion_matrix(classifier.predict(x), y, num_classes=len(np.unique(y)))
    acc = accuracy(cm)
    prec = precision(cm)
    rec = recall(cm)
    f1 = f1_score(cm)
    
    print('Accuracy: {:.2f}. (Min-class) Precision: {:.2f}. (Min-class) Recall: {:.2f}. (Min-class) F1-Score: {:.2f}. '.format(acc, prec, rec, f1))
    print('Confusion Matrix: \n', cm)


    
def multiclass(strategy, classifier, noise):
    np.random.seed(1)
    X, y = datasets.make_classification(n_samples=100, n_features=2, n_informative=2, n_redundant=0, n_repeated=0, 
                                        n_classes=3, n_clusters_per_class=1,
                                        class_sep=2, random_state=0)
    X += noise * np.random.randn(100, 2)
    
    if classifier == 'perceptron':
        base_classifier = SGDClassifier(loss='perceptron', alpha=0.001, random_state=0)
    elif classifier == 'svm':
        base_classifier = LinearSVC(random_state=0, loss='hinge')
    
    if strategy == 'OvO':
        classifier =  OneVsOneClassifier(base_classifier)
        colors = ['r', 'g', 'b']
    elif strategy == 'OvR' or strategy == 'OvA':
        classifier =  OneVsRestClassifier(base_classifier)
        colors = ['b', 'g', 'r']
    elif strategy == 'multi-class':
        classifier = base_classifier
        colors = ['b', 'g', 'r']

        
    
    classifier.fit(X, y)
    X0, X1 = X[:, 0], X[:, 1]
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    h = .02
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    fig = plt.subplot(111)
    plot_helpers.plot_contours(fig, classifier, xx, yy, cmap=plt.cm.jet, alpha=0.3)
    
    opt = {'marker': 'bs', 'label': '0', 'x_label': '$x$', 'y_label': '$y$', 'size': 8}
    plot_helpers.plot_data(X[np.where(y == 0)[0], 0], X[np.where(y == 0)[0], 1], fig=fig, options=opt)
    opt = {'marker': 'g*', 'label': '1', 'size': 8}
    plot_helpers.plot_data(X[np.where(y == 1)[0], 0], X[np.where(y == 1)[0], 1], fig=fig, options=opt)
    opt = {'marker': 'ro', 'label': '2', 'x_label': '$x$', 'y_label': '$y$', 'size': 8, 'legend': True}
    plot_helpers.plot_data(X[np.where(y == 2)[0], 0], X[np.where(y == 2)[0], 1], fig=fig, options=opt)
    

    def plot_hyperplane(coef, intercept, color):
        def line(x0):
            return (-(x0 * coef[0, 0]) - intercept[0]) / coef[0, 1]

        plt.plot([x_min, x_max], [line(x_min), line(x_max)],
                 ls="--", color=color)

    
    if hasattr(classifier, 'estimators_'):
        for i, (estimator, color) in enumerate(zip(classifier.estimators_, colors)):
            plot_hyperplane(estimator.coef_, estimator.intercept_, color)
            print('w{} = [{:.2f} {:.2f} {:.2f}]'.format(i, *estimator.coef_[0, :], estimator.intercept_[0]))
    else:
        for i, (coef, intercept, color) in enumerate(zip(classifier.coef_, classifier.intercept_, colors)):
            plot_hyperplane(coef[np.newaxis], intercept[np.newaxis], color)
            print('w{} = [{:.2f} {:.2f} {:.2f}]'.format(i, *coef[:], intercept))
            
    fig.set_xlim([x_min, x_max])
    fig.set_ylim([y_min, y_max]);
    
    print("")
    print_metrics(X, y, classifier)

noise_widget = ipywidgets.FloatSlider(value=0, min=0, max=2, step=0.1, continuous_update=False)
interact(multiclass, strategy=['OvA', 'OvO', 'multi-class'], classifier=['svm', 'perceptron'], noise=noise_widget); 